In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlsxwriter

from sklearn.preprocessing import OneHotEncoder

%matplotlib qt

# import module with layers
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model_module")

from adaptive_layer import *

In [2]:
tf.random.set_seed(5)
np.random.seed(5)

In [3]:
def illustrate_train(name, y_label, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    epochs = range(1, len(static_f)+1)
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    plt.title(name, fontsize="xx-large")

    ax.set_ylabel(y_label, fontsize="large")
    ax.set_xlabel('Epochs', fontsize="large")

    ax.plot(epochs, static_f, label='ReLU Function')
    ax.plot(epochs, adaptive_f, label='Swish Function')
    ax.plot(epochs, adaptive_nn, label='Adaptive NN')
    ax.plot(epochs, adaptive_conv_nn, label='Adaptive Conv NN')

    ax.legend()
    
    plt.grid()
    plt.show()

In [4]:
def illustrate_test(name, static_f, adaptive_f, adaptive_nn, adaptive_conv_nn):
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot()
    
    x = ["ReLU Function", "Swish Function", "Adaptive NN", "Adaptive Conv NN"]
    y = [static_f, adaptive_f, adaptive_nn, adaptive_conv_nn]
    
    plt.title(name, fontsize="xx-large")
    ax.set_ylabel("mean absolute error", fontsize="large")
    
    ax.bar(x, y)
    
    plt.grid()
    plt.show()

In [5]:
workbook = xlsxwriter.Workbook('carr_measurements.xlsx')

In [6]:
def save_data(name, static_f_train, adaptive_f_train, adaptive_nn_train, adaptive_conv_nn_train, 
              static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test):
    
    epochs = len(static_f_train[0][:])
    worksheet = workbook.add_worksheet(name)
    
    worksheet.write(1, 1, name + " Training Loss")
    worksheet.write(1, 7, name + " Training Accuracy")
    worksheet.write(1, 13, name + " Test Accuracy")
    
    worksheet.write(2, 1, "Epochs")
    worksheet.write(2, 2, "ReLU Function")
    worksheet.write(2, 3, "Swish Function")
    worksheet.write(2, 4, "Adaptive NN")
    worksheet.write(2, 5, "Adaptive Conv NN")
    
    worksheet.write(2, 7, "Epochs")
    worksheet.write(2, 8, "ReLU Function")
    worksheet.write(2, 9, "Swish Function")
    worksheet.write(2, 10, "Adaptive NN")
    worksheet.write(2, 11, "Adaptive Conv NN")
    
    worksheet.write(2, 13, "ReLU Function")
    worksheet.write(2, 14, "Swish Function")
    worksheet.write(2, 15, "Adaptive NN")
    worksheet.write(2, 16, "Adaptive Conv NN")
    
    worksheet.write(3, 13, static_f_test)
    worksheet.write(3, 14, adaptive_f_test)
    worksheet.write(3, 15, adaptive_nn_test)
    worksheet.write(3, 16, adaptive_conv_nn_test)
    
    for i in range(0, epochs):
        worksheet.write(3+i, 1, i+1)
        worksheet.write(3+i, 2, static_f_train[0][i])
        worksheet.write(3+i, 3, adaptive_f_train[0][i])
        worksheet.write(3+i, 4, adaptive_nn_train[0][i])
        worksheet.write(3+i, 5, adaptive_conv_nn_train[0][i])
        
        worksheet.write(3+i, 7, i+1)
        worksheet.write(3+i, 8, static_f_train[1][i])
        worksheet.write(3+i, 9, adaptive_f_train[1][i])
        worksheet.write(3+i, 10, adaptive_nn_train[1][i])
        worksheet.write(3+i, 11, adaptive_conv_nn_train[1][i])
    
    workbook.close()

In [7]:
# dataset

data = pd.read_csv('../datasets/car_data.csv')
cat_encoder = OneHotEncoder()
x = np.array([])
arr = np.array(data.Fuel_Type)
arr.shape = (-1, 1)
tmp = cat_encoder.fit_transform(arr).toarray()
x = np.append(x, tmp)
x.shape = (len(tmp), -1)
arr = np.array(data.Seller_Type)
arr.shape = (-1, 1)
tmp = cat_encoder.fit_transform(arr).toarray()
x = np.append(x, tmp, axis=-1)
x.shape = (len(tmp), -1)
arr = np.array(data.Transmission)
arr.shape = (-1, 1)
tmp = cat_encoder.fit_transform(arr).toarray()
x = np.append(x, tmp, axis=-1)
x.shape = (len(tmp), -1)
tmp = np.array(data.Year)
tmp.shape = (-1, 1)
tmp = (tmp - tmp.min())/(tmp.max() - tmp.min())
x = np.append(x, tmp, axis=-1)
x.shape = (len(tmp), -1)
tmp = np.array(data.Present_Price)
tmp.shape = (-1, 1)
tmp = (tmp - tmp.min())/(tmp.max() - tmp.min())
x = np.append(x, tmp, axis=-1)
x.shape = (len(tmp), -1)
tmp = np.array(data.Kms_Driven)
tmp.shape = (-1, 1)
tmp = (tmp - tmp.min())/(tmp.max() - tmp.min())
x = np.append(x, tmp, axis=-1)
x.shape = (len(tmp), -1)
tmp = np.array(data.Owner)
tmp.shape = (-1, 1)
x = np.append(x, tmp, axis=-1)
x.shape = (len(tmp), -1)
y = np.array(data.Selling_Price)
y.shape = (-1, 1)

x_train, x_test = x[:220], x[220:]
y_train, y_test = y[:220], y[220:]

In [8]:
EPOCHS = 200
name = "Car cost regression"

# measurement repetitions
flag = measurements = 5

static_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_f_hist = np.zeros(shape=[2, EPOCHS])
adaptive_nn_hist = np.zeros(shape=[2, EPOCHS])
adaptive_conv_nn_hist = np.zeros(shape=[2, EPOCHS])

static_f_test = 0
adaptive_f_test = 0
adaptive_nn_test = 0
adaptive_conv_nn_test = 0

In [89]:
static_f_model = keras.Sequential([
    keras.layers.Dense(150, activation="relu"),
    keras.layers.Dense(1, activation="linear")
])

In [90]:
adaptive_f_model = keras.Sequential([
    keras.layers.Dense(150, activation=tf.nn.swish),
    keras.layers.Dense(1, activation="linear")
])

In [91]:
# Adaptive Layer
adaptive_nn_model = AdaptiveModel([
    AdaptiveLayer(150, structure=[2, 1], inner_hidden_activation=tf.nn.leaky_relu), 
    AdaptiveLayer(1, structure=[4, 1], inner_hidden_activation=tf.nn.leaky_relu) 
])

In [92]:
# Adaptive Layer based on 1D Conv
adaptive_conv_nn_model = AdaptiveModel([
    AdaptiveLayerConv(150, structure=[16], noise=0, skip_w=0.9, 
                      inner_hidden_activation=tf.nn.leaky_relu), 
    AdaptiveLayerConv(1, structure=[32], noise=0, skip_w=0.9, 
                      inner_hidden_activation=tf.nn.relu) 
])

In [93]:
static_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="mse", metrics=['mean_absolute_error'])

In [94]:
adaptive_f_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="mse", metrics=['mean_absolute_error'])

In [95]:
adaptive_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="mse", metrics=['mean_absolute_error'])

In [96]:
adaptive_conv_nn_model.compile(optimizer=tf.optimizers.legacy.Adam(learning_rate=0.001), 
                       loss="mse", metrics=['mean_absolute_error'])

In [97]:
hist = static_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
4/4 [==============================] - 0s 961us/step - loss: 45.8216 - mean_absolute_error: 3.7837
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 44.3505 - mean_absolute_error: 3.6534
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 42.9230 - mean_absolute_error: 3.5622
Epoch 4/200
4/4 [==============================] - 0s 989us/step - loss: 41.4949 - mean_absolute_error: 3.4960
Epoch 5/200
4/4 [==============================] - 0s 976us/step - loss: 40.1558 - mean_absolute_error: 3.4285
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 38.8970 - mean_absolute_error: 3.3811
Epoch 7/200
4/4 [==============================] - 0s 889us/step - loss: 37.5133 - mean_absolute_error: 3.3139
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 36.1632 - mean_absolute_error: 3.2451
Epoch 9/200
4/4 [==============================] - 0s 903us/step - loss: 34.8396 - mean_absolute_error: 3.1752
Epoch 10/

In [98]:
static_f_hist[0][:] += np.array(hist.history["loss"])/measurements
static_f_hist[1][:] += np.array(hist.history["mean_absolute_error"])/measurements

In [99]:
hist = adaptive_f_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
4/4 [==============================] - 0s 1ms/step - loss: 49.6646 - mean_absolute_error: 4.2094
Epoch 2/200
4/4 [==============================] - 0s 961us/step - loss: 47.9569 - mean_absolute_error: 4.0205
Epoch 3/200
4/4 [==============================] - 0s 934us/step - loss: 46.3328 - mean_absolute_error: 3.8388
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 44.7650 - mean_absolute_error: 3.6850
Epoch 5/200
4/4 [==============================] - 0s 1ms/step - loss: 43.3218 - mean_absolute_error: 3.5775
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 42.0033 - mean_absolute_error: 3.5092
Epoch 7/200
4/4 [==============================] - 0s 947us/step - loss: 40.6184 - mean_absolute_error: 3.4293
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 39.3171 - mean_absolute_error: 3.3650
Epoch 9/200
4/4 [==============================] - 0s 909us/step - loss: 38.0790 - mean_absolute_error: 3.3158
Epoch 10/20

In [100]:
adaptive_f_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_f_hist[1][:] += np.array(hist.history["mean_absolute_error"])/measurements

In [101]:
hist = adaptive_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
4/4 [==============================] - 0s 1ms/step - loss: 48.8072 - mean_absolute_error: 4.0947
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 46.0110 - mean_absolute_error: 3.8131
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 43.2499 - mean_absolute_error: 3.6134
Epoch 4/200
4/4 [==============================] - 0s 2ms/step - loss: 40.4498 - mean_absolute_error: 3.4892
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 37.7953 - mean_absolute_error: 3.4116
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 35.3560 - mean_absolute_error: 3.3609
Epoch 7/200
4/4 [==============================] - 0s 2ms/step - loss: 32.7496 - mean_absolute_error: 3.2527
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 30.4375 - mean_absolute_error: 3.1923
Epoch 9/200
4/4 [==============================] - 0s 1ms/step - loss: 28.4584 - mean_absolute_error: 3.1970
Epoch 10/200
4/4 [=

In [102]:
adaptive_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_nn_hist[1][:] += np.array(hist.history["mean_absolute_error"])/measurements

In [103]:
hist = adaptive_conv_nn_model.fit(x_train, y_train, epochs=EPOCHS, batch_size=64)

Epoch 1/200
4/4 [==============================] - 0s 1ms/step - loss: 49.1058 - mean_absolute_error: 4.1315
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 47.7620 - mean_absolute_error: 3.9812
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 46.1799 - mean_absolute_error: 3.8137
Epoch 4/200
4/4 [==============================] - 0s 2ms/step - loss: 44.0692 - mean_absolute_error: 3.6491
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 41.3553 - mean_absolute_error: 3.4966
Epoch 6/200
4/4 [==============================] - 0s 2ms/step - loss: 38.0675 - mean_absolute_error: 3.4018
Epoch 7/200
4/4 [==============================] - 0s 2ms/step - loss: 33.7314 - mean_absolute_error: 3.2472
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 29.6493 - mean_absolute_error: 3.2298
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 27.0964 - mean_absolute_error: 3.5041
Epoch 10/200
4/4 [=

In [104]:
adaptive_conv_nn_hist[0][:] += np.array(hist.history["loss"])/measurements
adaptive_conv_nn_hist[1][:] += np.array(hist.history["mean_absolute_error"])/measurements

In [105]:
static_f_test += static_f_model.evaluate(x_test, y_test)[1]/measurements

3/3 [==============================] - 0s 1ms/step - loss: 2.1518 - mean_absolute_error: 1.0822


In [106]:
adaptive_f_test += adaptive_f_model.evaluate(x_test, y_test)[1]/measurements

3/3 [==============================] - 0s 1ms/step - loss: 3.5035 - mean_absolute_error: 1.4516


In [107]:
adaptive_nn_test += adaptive_nn_model.evaluate(x_test, y_test)[1]/measurements

3/3 [==============================] - 0s 1ms/step - loss: 0.9754 - mean_absolute_error: 0.7282


In [108]:
adaptive_conv_nn_test += adaptive_conv_nn_model.evaluate(x_test, y_test)[1]/measurements
flag -= 1
if(flag == 0):
    print("---------FINISH---------")

3/3 [==============================] - 0s 1ms/step - loss: 0.9256 - mean_absolute_error: 0.7513
---------FINISH---------


In [109]:
print("Static Function model\nTest mean_absolute_error : ", static_f_test)
print("\nAdaptive Function model\nTest mean_absolute_error : ", adaptive_f_test)
print("\nAdaptive NN model\nTest mean_absolute_error : ", adaptive_nn_test)
print("\nAdaptive Conv NN model\nTest mean_absolute_error : ", adaptive_conv_nn_test)

Static Function model
Test mean_absolute_error :  1.0703433990478515

Adaptive Function model
Test mean_absolute_error :  1.4475241422653198

Adaptive NN model
Test mean_absolute_error :  0.7668860793113708

Adaptive Conv NN model
Test mean_absolute_error :  0.7334017753601074


In [110]:
illustrate_train(name+" Training Loss", "Loss", static_f_hist[0][:], 
                 adaptive_f_hist[0][:], adaptive_nn_hist[0][:], adaptive_conv_nn_hist[0][:])

In [111]:
illustrate_train(name + " Training Mean Absolute Error", "mean absolute error", static_f_hist[1][:], 
                 adaptive_f_hist[1][:], adaptive_nn_hist[1][:], adaptive_conv_nn_hist[1][:])

In [112]:
illustrate_test(name + " Test", 
                static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)

In [113]:
save_data(name, static_f_hist, adaptive_f_hist, adaptive_nn_hist, 
          adaptive_conv_nn_hist, static_f_test, adaptive_f_test, adaptive_nn_test, adaptive_conv_nn_test)